## Creating and Saving COCO Datasets from Photopea Annotations

In [ ]:
import ceruleanml.data as data
from pathlib import Path
import skimage.io as skio
import os

ml_data_path = os.path.join(os.path.abspath(os.getcwd()),"../../data/cv2_transfer/outputs/skytruth_annotation/first_phase/")
coco_outdir = "./"
os.makedirs(coco_outdir, exist_ok=True)
os.makedirs("untiled_images/", exist_ok=True)
path = Path(ml_data_path)
class_folders = list(path.glob("*/"))

In [ ]:
info = {
    "description": "Cerulean Dataset V2",
    "url": "none",
    "version": "1.0",
    "year": 2021,
    "contributor": "Skytruth",
    "date_created": "2022/2/23"
}

licenses = [
    {
        "url": "none",
        "id": 1,
        "name": "CeruleanDataset V2"
    }
]
categories = [{"supercategory":"slick", "id":1,"name":"infra_slick"},
              {"supercategory":"slick", "id":2,"name":"natural_seep"},
              {"supercategory":"slick", "id":3,"name":"coincident_vessel"},
              {"supercategory":"slick", "id":4,"name":"recent_vessel"},
              {"supercategory":"slick", "id":5,"name":"old_vessel"},
              {"supercategory":"slick", "id":6,"name":"ambiguous"}]

coco_output = {
    "info": info,
    "licenses": licenses,
    "images": [],
    "annotations": [],
    "categories": categories
}

### With Tiling

In [ ]:
coco_tiler = data.COCOtiler("untiled_images/", coco_output)

for class_folder in class_folders:
    for scene_folder in list(class_folder.glob("*")):
        assert "S1" in str(scene_folder)
        layer_pths = list(scene_folder.glob("*"))
        coco_tiler.save_background_img_tiles(layer_pths)
        coco_tiler.create_coco_from_photopea_layers(layer_pths, coco_output)

coco_tiler.save_coco_output("./instances_slicks_test_v2.json")

### No tiling

In [ ]:
import tqdm

coco_tiler = data.COCOtiler("untiled_images/", coco_output)
coco_tiler.copy_background_images(class_folders)

for class_folder in class_folders:
    for scene_folder in tqdm.tqdm(list(class_folder.glob("*"))):
        assert "S1" in str(scene_folder)
        layer_pths = list(scene_folder.glob("*"))
        
        coco_tiler.create_coco_from_photopea_layers_no_tile(layer_pths, coco_output)

coco_tiler.save_coco_output("./instances_slicks_test_v2_whole_image.json")

!cp "./instances_slicks_test_v2_whole_image.json" "../../data/cv2_transfer/"

!cp "./instances_slicks_test_v2_whole_image_small.json" "../../data/cv2_transfer/"